In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
from pathlib import Path

# check if workding_dir is in local variables
if "workding_dir" not in locals():
    workding_dir = str(Path.cwd().parent)

os.chdir(workding_dir)
sys.path.append(workding_dir)
print("workding dir:", workding_dir)

In [ ]:
from dotenv import find_dotenv, load_dotenv

found_dotenv = find_dotenv(".env")

if len(found_dotenv) == 0:
    found_dotenv = find_dotenv(".env.example")
print(f"loading env vars from: {found_dotenv}")
load_dotenv(found_dotenv, override=True)

In [ ]:
from llm_toolkit.data_utils import *

data_path = "dataset/amazon_reviews.csv"
results_path = "results/amazon_reviews_results.csv"
num_ctx = os.getenv("NUM_CTX")
data_path, results_path, num_ctx

In [ ]:
import pandas as pd

df = pd.read_csv(results_path)
df.info()

In [ ]:
result_col_start_idx = 3
selected_result_col_start_idx = 3

In [ ]:
columns = {
    col: col.replace("_8k", "").replace("-chat", "-v3").replace("-reasoner", "-r1")
    for col in df.columns
}
df.rename(columns=columns, inplace=True)

In [ ]:
def get_key(model_name):
    parts = model_name.split("/")
    key = f"{model_orders[parts[0]]:03d}/{parts[1]}"
    # print(key)
    return key


result_cols = [
    col
    for col in df.columns[selected_result_col_start_idx:].tolist()
    if col.split("/")[0] in model_orders.keys()
]
result_cols.sort(key=get_key)
result_cols

In [ ]:
df = df[df.columns[:result_col_start_idx].tolist() + result_cols]

In [ ]:
df.info()

In [ ]:
datasets = prepare_dataset(data_path, "Text", "Review-sentiment")

In [ ]:
metrics = get_metrics(
    df,
    result_col_start_idx=result_col_start_idx,
    mean_eval_time=True,
    train_dataset=datasets["train"],  # , debug=True
)
metrics

In [ ]:
metrics_df = metrics.copy()
models = metrics_df["model"].unique()
models

In [ ]:
from llm_toolkit.data_utils import *

plot_metrics_vs_shots(
    metrics_df,
    models,
    markers,
    ["f1", "accuracy"],
    ["F1 Score", "Accuracy"],
    # ylimits=(0.6, 0.8),
    # log_scales=[False, True],
    bbox_to_anchor=(0.5, -0.5),
    sync_y_axis=True,
)

In [ ]:
from llm_toolkit.data_utils import *

plot_metrics_vs_shots(
    metrics_df,
    models,
    markers,
    ["f1_5_level", "accuracy_5_level"],
    ["F1 Score", "Accuracy"],
    # ylimits=(0.6, 0.8),
    # log_scales=[False, True],
    bbox_to_anchor=(0.5, -0.5),
    sync_y_axis=True,
)

In [ ]:
from llm_toolkit.data_utils import *

plot_metrics_vs_shots(
    metrics_df,
    models,
    markers,
    ["f1_5_level", "eval_time"],
    ["F1 Score", "Mean Eval Time (s)"],
    # ylimits=(0.6, 0.8),
    # log_scales=[False, True],
    bbox_to_anchor=(0.5, -0.5),
)

In [ ]:
from llm_toolkit.data_utils import *

plot_metrics_vs_shots(
    metrics_df,
    models,
    markers,
    ["f1_5_level", "eval_speed"],
    ["F1 Score", "Throughput (token/s)"],
    # ylimits=(0.6, 0.8),
    # log_scales=[False, True],
    bbox_to_anchor=(0.5, -0.5),
)

In [ ]:
metrics["model"] = metrics.apply(
    lambda x: x["model"] + f"\n({x['shots']}-shot)", axis=1
)

In [ ]:
metrics

In [ ]:
from llm_toolkit.data_utils import plot_barcharts_for_dual_metrics

# Call the function to plot
plot_barcharts_for_dual_metrics(metrics)

In [ ]:
plot_barcharts_for_dual_metrics(
    metrics,
    title="Aggregate Level F1 and Accuracy Scores Across Models",
    ylabels=("F1 Score (%)", "Accuracy (%)"),
    columns=("f1", "accuracy"),
    use_percentage=(True, True),
    decimal_places=(2, 2),
    y_limit_offsets=(30, 30),
)

In [ ]:
plot_barcharts_for_dual_metrics(
    metrics,
    title="Multi Level F1 and Accuracy Scores Across Models",
    columns=("f1_5_level", "accuracy_5_level"),
    ylabels=("F1 Score (%)", "Accuracy (%)"),
    use_percentage=(True, True),
    decimal_places=(2, 2),
    y_limit_offsets=(30, 30),
)

In [ ]:
top_metrics_df = get_top_metrics_df(metrics_df, col="f1_5_level")
top_metrics_df["model"] = top_metrics_df.apply(
    lambda x: x["model"] + f"\n({x['shots']}-shot)", axis=1
)
top_metrics_df

In [ ]:
from llm_toolkit.data_utils import plot_barcharts_for_dual_metrics

# Call the function to plot
plot_barcharts_for_dual_metrics(top_metrics_df, decimal_places=(3, 0))

In [ ]:
plot_barcharts_for_dual_metrics(
    top_metrics_df,
    title="Aggregate Level F1 and Accuracy Scores Across Models",
    columns=("f1", "accuracy"),
    ylabels=("F1 Score (%)", "Accuracy (%)"),
    use_percentage=(True, True),
    decimal_places=(2, 2),
    y_limit_offsets=(30, 30),
)

In [ ]:
plot_barcharts_for_dual_metrics(
    top_metrics_df,
    title="Multi Level F1 and Accuracy Scores Across Models",
    columns=("f1_5_level", "accuracy_5_level"),
    ylabels=("F1 Score (%)", "Accuracy (%)"),
    use_percentage=(True, True),
    decimal_places=(2, 2),
    y_limit_offsets=(30, 30),
)

In [ ]:
metrics_df.to_csv(results_path.replace(".csv", "_metrics.csv"), index=False)

In [ ]:
df["Review-sentiment"].value_counts()

In [ ]:
labels = ["Strongly Positive", "Positive", "Neutral", "Negative", "Strongly Negative"]

In [ ]:
plot_confusion_matrix(
    df,
    "Review-sentiment",
    "gpt-4o/shots-40(2.350)",
    "gpt-4o/shots-40(2.350)",
    labels,
    vmax=320,
    title="Amazon Reviews Dataset - Confusion Matrix - GPT-4o",
)

In [ ]:
plot_confusion_matrix(
    df,
    "Review-sentiment",
    "deepseek-r1/shots-30(10.097)",
    "gpt-4o/shots-40(2.350)",
    labels,
    vmax=320,
    title="Amazon Reviews Dataset - Confusion Matrix - DeepSeek-R1",
)

In [ ]:
df2 = analyze_confusion_cases(df, "Positive", "Neutral", datasets["train"])

In [ ]:
df2[
    [
        "Text",
        "Review-sentiment",
        "DeepSeek-sentiment",
        "Gemini-sentiment",
        "GPT-4o-sentiment",
    ]
]

In [ ]:
df3 = analyze_confusion_cases(df, "Positive", "Negative", datasets["train"])
df3[
    [
        "Text",
        "Review-sentiment",
        "DeepSeek-sentiment",
        "Gemini-sentiment",
        "GPT-4o-sentiment",
    ]
]

In [ ]:
df4 = analyze_confusion_cases(df, "Neutral", "Positive", datasets["train"])
# df4[
#     [
#         "Text",
#         "Review-sentiment",
#         "DeepSeek-sentiment",
#         "Gemini-sentiment",
#         "GPT-4o-sentiment",
#     ]
# ]

In [ ]:
df5 = analyze_confusion_cases(df, "Neutral", "Negative", datasets["train"])
df5[
    [
        "Text",
        "Review-sentiment",
        "DeepSeek-sentiment",
        "Gemini-sentiment",
        "GPT-4o-sentiment",
    ]
]

In [ ]:
df6 = analyze_confusion_cases(df, "Negative", "Positive", datasets["train"])
# df6[
#     [
#         "Text",
#         "Review-sentiment",
#         "DeepSeek-sentiment",
#         "Gemini-sentiment",
#         "GPT-4o-sentiment",
#     ]
# ]

In [ ]:
df7 = analyze_confusion_cases(df, "Negative", "Neutral", datasets["train"])
df7[
    [
        "Text",
        "Review-sentiment",
        "DeepSeek-sentiment",
        "Gemini-sentiment",
        "GPT-4o-sentiment",
    ]
]

In [ ]:
df7.info()

In [ ]:
# Select only the specified columns
columns = [
    "Text",
    "Review-sentiment",
    "Review-basic-sentiment",
    "DeepSeek-sentiment",
    "Gemini-sentiment",
    "GPT-4o-sentiment",
    "Content",
    "Reasoning-content",
    "Gemini-content",
    "Gemini-reasoning-content",
    "GPT-4o-Content",
]

# Concatenate df2 and df7 with only the specified columns
result = pd.DataFrame(columns=columns)
for df in [df2, df3, df4, df4, df6, df7]:
    if len(df) > 0:
        result = pd.concat([result, df[columns]])
result

In [ ]:
result.to_csv(results_path.replace(".csv", "_confusion_cases.csv"), index=False)

In [ ]:
df8 = analyze_confusion_cases(df, "Neutral", "Neutral", datasets["train"])
df8[
    [
        "Text",
        "Review-sentiment",
        "DeepSeek-sentiment",
        "Gemini-sentiment",
        "GPT-4o-sentiment",
    ]
]

In [ ]:
df_n_n = df[df["Review-sentiment"] == "Neutral"][df["DeepSeek-sentiment"] == "Neutral"]
len(df_n_n)

In [ ]:
df = pd.read_csv(results_path.replace(".csv", "_confusion_cases.csv"))
df.info()

In [ ]:
# Select only the specified columns
columns = [
    "Text",
    "Review-sentiment",
    "Review-basic-sentiment",
    "DeepSeek-sentiment",
    "Gemini-sentiment",
    "GPT-4o-sentiment",
    "Content",
    "Reasoning-content",
    "Gemini-content",
    "Gemini-reasoning-content",
    "GPT-4o-Content",
]

df = pd.concat([df, df8[columns]])
df.info()

In [ ]:
df.to_csv(results_path.replace(".csv", "_confusion_cases.csv"), index=False)

In [ ]:
df_neutral_negative = df[df["Review-sentiment"] == "Neutral"][
    df["DeepSeek-sentiment"] == "Neutral"
][df["GPT-4o-sentiment"] == "Negative"]
len(df_neutral_negative)

In [ ]:
print_row_details(df_neutral_negative)

In [ ]:
df_neutral_neutral = df[df["Review-sentiment"] == "Neutral"][
    df["DeepSeek-sentiment"] == "Neutral"
][df["GPT-4o-sentiment"] == "Neutral"]
len(df_neutral_neutral)

In [ ]:
print_row_details(df_neutral_neutral)

In [ ]:
print_row_details(df_neutral_neutral, [1])

In [ ]:
df_sp_n = df[df["Review-sentiment"] == "Strongly Positive"][
    df["DeepSeek-sentiment"] == "Negative"
]

In [ ]:
print_row_details(df_sp_n)  # , range(len(df_sp_n)))

In [ ]:
df_sn_n = df[df["Review-sentiment"] == "Strongly Negative"][
    df["DeepSeek-sentiment"] == "Neutral"
]

In [ ]:
print_row_details(df_sn_n)  # , range(len(df_sn_n)))